# Calculate indices from HYPERSPECTRAL image

#### Install libraries

In [ ]:
# !pip install rasterio matplotlib pyproj

#### Import libraries and read hyperspectral image (.tiff)

In [ ]:
from math import log
import numpy as np
import pandas as pd
import rasterio
from rasterio.plot import show
from rasterio.transform import rowcol
import matplotlib.pyplot as plt
from pyproj import Proj, transform
import openpyxl
import shutil

# Path to the .tiff file
working_dir = "C:/Users/User/OneDrive - UVa/1_ASIGNATURAS/TFM/1_DataProcessing/"
tiff_file_orig = working_dir + "2_SatellitalImages/2_HYP/inicial_2_mtotal.tiff"
tiff_file = working_dir + "2_SatellitalImages/2_HYP/z_CoRegisteredImage/imagen_coregistered_global_191band.tif"

#### Open hyperspectral image and classify wavelengths by landsat approach

Landsat 9 Band
'SWIR 2' 2110 - 2290
'SWIR 1' 1570 - 1650
'Cirrus' 1360 - 1380
'NIR' 850 - 880
'Red Edge' 715 - 720
'Red' 640 - 670
'Green' 530 - 590
'Blue' 450 - 510
'Coastal Aerosol' 430 - 450

In [ ]:
def open_tiff(tiff_file):
    # Open the GeoTIFF file
    with rasterio.open(tiff_file) as src:
        # Read the custom metadata (wavelengths)
        wavelengths = src.tags().get('Wavelengths', 'No wavelengths information found')
        if wavelengths != 'No wavelengths information found':
            wavelengths = eval(wavelengths)  # Convert string representation back to list

        # Convert a list of strings to a list of numbers
        wavelength = []
        for wv in wavelengths:
            wavelength.append(float(wv))
    return wavelength

def classify_wavelength(wl):
    if 1900 <= wl < 2500:
        return 'SWIR 2'
    elif 1050 <= wl < 1900:
        return 'SWIR 1'
    elif 750 <= wl < 1050:
        return 'NIR'
    elif 700 <= wl < 750:
        return 'Red Edge'
    elif 600 <= wl < 700:
        return 'Red'
    elif 500 <= wl < 600:
        return 'Green'
    elif 400 <= wl < 500:
        return 'Blue'
    else:
        return '***'
    
def create_band_list(wavelength):
    band_lst = []
    count = 1
    for wl in wavelength:
        bandL9 = classify_wavelength(wl)
        band_classL9_tuples = (count, bandL9, wl)
        band_lst.append(band_classL9_tuples)
        count += 1
    return band_lst

# Run the classification
wavelength = open_tiff(tiff_file_orig)
band_lst = create_band_list(wavelength)
for m in band_lst:
    print(m)

#### Vegetation Indices Formulas

In [ ]:
# Normalized Difference Vegetation Index: NDVI
def calculate_ndvi(red, nir):
    ndvi = (nir - red) / (nir + red)
    return ndvi

# Normalized Burn Ratio: NBR
def calculate_nbr(nir, swir2):
    nbr = (nir - swir2) / (nir + swir2)
    return nbr

# Normalized Difference Moisture Index: NDMI
def calculate_ndmi(nir, swir1):
    ndmi = (nir - swir1) / (nir + swir1)
    return ndmi

# Difference Vegetation Index: DVI
def calculate_dvi(red, nir):
    dvi = nir - red
    return dvi

# Green Difference Vegetation Index: DVIGRE o GDVI
def calculate_dvigre(green, nir):
    dvigre = nir - green
    return dvigre

# Red Difference Vegetation Index: DVIRED
def calculate_dvired(redge, nir):
    dvired = nir - redge
    return dvired

# Chlorophyll Index With Red Edge: CIREDGE
def calculate_ciredge(redge, nir):
    ciredge = (nir / redge) - 1
    return ciredge

# Chlorophyll Index With Green: CIGREEN
def calculate_cigreen(green, nir):
    cigreen = (nir / green) - 1
    return cigreen

# Infrared Percentage Vegetation Index: IPVI
def calculate_ipvi(red, nir):
    ipvi = nir / (nir + red)
    return ipvi

# Near-Infrared Reflectance of Vegetation: NIRV
def calculate_nirv(red, nir):
    nirv = nir * ((nir - red) / (nir + red))
    return nirv

# Modified Non-Linear Index: MNLI
def calculate_mnli(nir, red): 
    mnli = 1.5 * (nir ** 2 - red) / (nir ** 2 + red + 0.5)
    return mnli

# Non-Linear Index: NLI
def calculate_nli(nir, red): 
    nli = (nir ** 2 - red) / (nir ** 2 + red)
    return nli

# Wide Dynamic Range Vegetation Index: WDRVI
def calculate_wdrvi(nir, red): 
    wdrvi = (0.2 * nir - red) / (0.2 * nir + red)
    return wdrvi
##########################################################################
# Normalized Difference Red Edge Index: NDRE
def calculate_ndre(redge, nir):
    ndre = (nir - redge) / (nir + redge)
    return ndre

# Burn Area Index: BAI
def calculate_bai(redge, nir):
    bai = 1 / (((redge - 0.1) ** 2) + ((nir - 0.06) ** 2))
    return bai

# Structural Vegetation Index: SVI
def calculate_svi(redge, red):
    svi = (redge - red) / (redge + red)
    return svi

# Modified Red-Edge Simple Ratio: MRESR
def calculate_mresr(redge, nir):
    mresr = nir / redge
    return mresr

In [ ]:
# Three-Band Difference Vegetation Index: TBDVI
def calculate_tbdvi(red, nir, swir1):
    tbdvi = nir - (red - swir1) / 2
    return tbdvi

# Enhanced Vegetation Index: EVI
def calculate_evi(blue, red, nir):
    evi = 2.5 * ((nir - red)/(nir + 6 * red - 7.5 * blue + 1))
    return evi

# Excess Green Index: EXG2
def calculate_exg2(blue, green, red):
    exg2 = 2 * green - red - blue
    return exg2

# Red Light Normalized Value: NRI
def calculate_nri(blue, green, red):
    nri = red / (red + green + blue)
    return nri

# Green Light Normalized Value: NGI
def calculate_ngi(blue, green, red):
    ngi = green / (red + green + blue)
    return ngi

# Green Normalized Difference Vegetation Index: GNDVI
def calculate_gndvi(green, redge, nir):
    gndvi = (nir - green) / (nir + redge)
    return gndvi

# Enhances Normalized Difference Vegetation Index: ENDVI
def calculate_endvi(redge, green, blue): 
    endvi = (redge + green - 2 * blue) / (redge + green + 2 * blue)
    return endvi

# Modified Red Green- Blue Vegetation Index: MRGBVI
def calculate_mrgbvi(redge, green, blue): 
    mrgbvi = (redge + 2 * green - 2 * blue) / (redge + 2 * green + 2 * blue)
    return mrgbvi

# Nitrogen Reflectance Index: NREI
def calculate_nrei(green, redge, nir):
    nrei = redge / (redge + nir + green)
    return nrei

##########################################################################
# Red Edge Position Index: REP
def calculate_rep(red, redge, nir):
    rep = 700 + 40 * ((red + nir)/(2 - redge))
    return rep

In [ ]:
def normalized_band (red, green, blue):
    R = red / (red + green + blue)
    G = green / (red + green + blue)
    B = blue / (red + green + blue)
    return R, G, B

# Excess Green Index: EXG
def calculate_exg(red, green, blue):
    R,G,B = normalized_band(red, green, blue)
    exg = 2 * G - R - B
    return exg

# Excess Blue Index: EXB
def calculate_exb(red, green, blue):
    R,G,B = normalized_band(red, green, blue)
    exb = 1.4 * R - G
    return exb

# Excess Red Index: EXR
def calculate_exr(red, green, blue):
    R,G,B = normalized_band(red, green, blue)
    exr = 1.4 * B - G
    return exr

# Red / Green Ratio: RGR
def calculate_rgr(red, green, blue):
    R,G,B = normalized_band(red, green, blue)
    rgr = R / G
    return rgr

# Blue / Green Ratio: BGR
def calculate_bgr(red, green, blue):
    R,G,B = normalized_band(red, green, blue)
    bgr = B / G
    return bgr

# Normalized Green-Red Difference Index: NGRDI
def calculate_ngrdi(red, green, blue):
    R,G,B = normalized_band(red, green, blue)
    ngrdi = (G - R) / (G + R)
    return ngrdi

# Normalized Green-Blue Difference Index: NGBDI
def calculate_ngbdi(red, green, blue):
    R,G,B = normalized_band(red, green, blue)
    ngbdi = (G - B) / (G + B)
    return ngbdi

# Modified Green-Red Vegetation Index: MGRVI
def calculate_mgrvi(red, green, blue):
    R,G,B = normalized_band(red, green, blue)
    mgrvi = (G**2 - R**2) / (G**2 + R**2)
    return mgrvi

# Red Green- Blue Vegetation Index: RGBVI
def calculate_rgbvi(red, green, blue):
    R,G,B = normalized_band(red, green, blue)
    rgbvi = (G**2 - R * B) / (G**2 + R * B)
    return rgbvi

# Green Leaf Index: GLI
def calculate_gli(red, green, blue):
    R,G,B = normalized_band(red, green, blue)
    gli = (2*G - R - B) / (2*G + R + B)
    return gli

# Color Index of Vegetation Extraction: CIVE
def calculate_cive(red, green, blue):
    R,G,B = normalized_band(red, green, blue)
    cive = 0.441*R - 0.881*G + 0.385*R + 18.78745
    return cive

# Excess Green Minus Excess Red Index: ExGR
def calculate_exgr(red, green, blue):
    exg = calculate_exg(red, green, blue)
    exr = calculate_exr(red, green, blue)
    exgr = exg - exr
    return exgr

# Improved Red Green- Blue Vegetation Index: IRGBVI
def calculate_irgbvi(red, green, blue):
    R,G,B = normalized_band(red, green, blue)
    irgbvi = (5*G**2 - 2*R**2 - 5*B**2) / (5*G**2 + 2*R**2 + 5*B**2)
    return irgbvi

In [ ]:
# Model 1 in Vegetation Index
def calculate_vi_model1(band_a, band_b, band_c, band_d):
    vi_model1 = band_a / band_b
    return vi_model1
    # Greenness Index: G = r554/r677
    # Simple R. Pigment Ind.: SRPI = r430/r680
    # Lichtenthaler Index 2: Lic2 = r440 / r690
    # Carter Index 1: Ctr1 = r695 / r420
    # Carter Index 2: Ctr2 = r695 / r760
    # Vogelmann Index 1: Vog1 = r740 / r720
    # Gitelson and Merzlyak Index 1: GM1 = r750 / r550
    # Gitelson and Merzlyak Index 2: GM2 = r750 / r700
    # Zarco-Tejada & Miller: ZM = r750 / r710
    # Fluorescence Ratio Index 1: FRI1 = r740 / r800
    # Fluorescence Ratio Index 2: FRI2 = r690 / r600
    # Simple Ratio Index: SR = r800 / r670
    # Water Index: WI1 = r900 / r970
    # Water Index: WI2 = r1600 / r820
    # Moisture Stress Index: MSI = r1599 / r819
    # Ratio Vegetation Index: RVI = r887 / r678
    # Green Ratio Vegetation Index: GRVI = r865/r550
    # Green Red Ratio Vegetation Index: GR = r550/r650
    # Green Blue Ratio Vegetation Index: GB = r550/r485
    # VI8 = r998 / r713
    # Water Band Index: WBI = wl_970 / wl_900

# Model 2a in Vegetation Index
def calculate_vi_model2a(band_a, band_b, band_c, band_d):
    vi_model2a = (band_a - band_b) / (band_a + band_b)
    return vi_model2a
    # Normalized Phaeophytinization Index: NPQI = (r415 - r435) / (r415 + r435)
    # Photochemical Reflectance Index 1: PRI1 = (r528 - r567) / (r528 + r567)
    # Normalized Pigment Chlorophyll Index: NPCI = (r680 - r430) / (r680 + r430)
    # Lichtenthaler Index 1: Lic1 = (r800 - r680) / (r800 + r680)
    # Visible Atmospherically Resistant Index: VARI = (r550 - r670) / (r550 + r670)
    # Normalized Difference Water Index: NDWI = (r800 - r1600) / (r800 + r1600)
    # Normalized Difference Water Index 2: NDWI2 = (r857 - r1241) / (r857 + r1241)
    # Depth Water Index: DWI = (r816 - r2218) / (r816 + r2218)
    # Hyperspectral Fire Detection Index: HFDI = (r2430 - r2060) / (r2430 + r2060)
    # VI7 = (r998 - r713) / (r998 + r713)
    # Normalized Difference Vegetation Index red-edge 1: NDVIre1 = (r833 - r699) / (r833 + r699)
    # Normalized Difference Vegetation Index red-edge 1: NDVIre1n = (r865 - r699) / (r865 + r699)
    # Normalized Difference Vegetation Index red-edge 2: NDVIre2 = (r833 - r729) / (r833 + r729)
    # Normalized Difference Vegetation Index red-edge 2: NDVIre2n = (r865 - r729) / (r865 + r729)
    # Normalized Difference Vegetation Index red-edge 3: NDVIre3 = (r833 - r780) / (r833 + r780)
    # Normalized Difference Vegetation Index red-edge 3: NDVIre3n = (r865 - r780) / (r865 + r780)
    # Normalized Difference red-edge 1: NDre1 = (r729 - r699) / (r729 + r699)
    # Normalized Difference red-edge 2: NDre2 = (r780 - r699) / (r780 + r699)
    # Leaf Water Vegetation Index 1: LWVI1 = (r1099 - r988) / (r1099 + r988)
    # Leaf Water Vegetation Index 2: LWVI2 = (r1099 - r1207) / (r1099 + r1205)
    # Normalized Difference Infrared Index: NDII = (r823 - r1646) / (r823 + r1646)

# Model 2b in Vegetation Index
def calculate_vi_model2b(band_a, band_b, band_c, band_d):
    vi_model2b = (band_a - band_b) / (band_a + band_c)
    return vi_model2b
    # Structure Intensive Pigment Index: SIPI = (r800 - r450) / (r800 + r650)
    # Meris Terrestrial Chlorophyll Index: MTCI = (r865 - r717) / (r865 - r650)
    # Leaf Chlorophyll Index: LCI = (r855 - r708) / (r855 + r679)

# Model 2c in Vegetation Index
def calculate_vi_model2c(band_a, band_b, band_c, band_d):
    vi_model2c = (band_a - band_b) / (band_c - band_b)
    return vi_model2c
    # Mangrove Vegetation Index: MVI = (r842 - r561) / (r1610 - r561)

# Model 2d in Vegetation Index
def calculate_vi_model2d(band_a, band_b, band_c, band_d):
    vi_model2d = (band_a - band_b) / (band_c + band_d)
    return vi_model2d
    # Vogelmann Index 2: Vog2 = (r734 - r747) / (r715 + r726)
    # Vogelmann Index 3: Vog3 = (r734 - r747) / (r715 + r720)
    # Disease Water Stress Index: DWSI = (r801 - r546) / (r1656 + r879)

# Model 2e in Vegetation Index
def calculate_vi_model2e(band_a, band_b, band_c, band_d):
    vi_model2e = (band_a ** 2- band_b ** 2) / (band_a ** 2 + band_b ** 2)
    return vi_model2e
    # VI5 = (r513 ** 2 - r504 **2) / (r513 ** 2 + r504 ** 2)

# Model 2f in Vegetation Index
def calculate_vi_model2f(band_a, band_b, band_c, band_d):
    vi_model2f = (band_a- band_b) / (band_c - band_b)
    return vi_model2f
    # Simple Ratio red-edge 1: SRre1 = (r729 - r441) / (r699 - r441)
    # Simple Ratio red-edge 2: SRre2 = (r780 - r441) / (r699 - r441)
    # Structure Independent Pigment Index: SIPI1 = (r445 - r800) / (r670 + r800)

# Model 3a in Vegetation Index
def calculate_vi_model3a(band_a, band_b, band_c, band_d):
    vi_model3a = (band_a / band_b) / ((band_a / band_b)**0.5)
    return vi_model3a
    # Renormalized Difference Vegetation Index: RDVI = (r800 / r670) / ((r800 / r670)**0.5)
    # Renormalized Difference Vegetation Index: RDVI2 = (r865 / r670) / ((r865 / r670)**0.5)
    # Red Difference Vegetation Index With Red Edge: RDVIREG = (r865 - r717) / ((r865 + r717) ** 0.5)

# Model 3b in Vegetation Index
def calculate_vi_model3b(band_a, band_b, band_c, band_d):
    vi_model3b = (band_a / (band_b - 1)) / ((band_a / (band_b + 1))**0.5)
    return vi_model3b
    # Modified Simple Ratio: MSR = (r865 / (r660 - 1))/(r865 / (r660 + 1)) ** 0.5
    # Modified Simple Ratio With Red Edge: MSRREG = (r865 / (r717 - 1))/(r865 / (r717 + 1)) ** 0.5

# Model 4 in Vegetation Index
def calculate_vi_model4(band_a, band_b, band_c, band_d): 
    vi_model4 = ((band_a / band_b) - 1) / (((band_a / band_b) + 1)**0.5)
    return vi_model4
    # Modified Simple Ratio Index: MSR2 = ((r800 / r670) - 1) / (((r800 / r670) + 1)**0.5)

# Model 5a in Vegetation Index
def calculate_vi_model5a(band_a, band_b, band_c, band_d): 
    vi_model5a = (1 + 0.16) * (band_a - band_b) / (band_a + band_b + 0.16)
    return vi_model5a
    # Optimized Soil-Adjusted Vegetation Index: OSAVI = (1 + 0.16) * (r800 - r670) / (r800 + r670 + 0.16)
    # Soil-Adjusted Vegetation Index: SAVI = (1 + 0.16) * (r887 - r678) / (r887 + r678 + 0.16)
    # Optimized Soil-adjusted Vegetation Index With Red Edge: OSAVIREG = (1 + 0.16) * (r865 - r717) / (r865 + r717 + 0.16)

# Model 5b in Vegetation Index
def calculate_vi_model5b(band_a, band_b, band_c, band_d): 
    vi_model5b = 1.5 * (band_a - band_b) / (band_a + band_b + 0.5)
    return vi_model5b
    # Green Soil Adjusted Vegetation Index: GSAVI = 1.5 * (r800 - r560) / (r800 + r560 + 0.5)

# Model 6 in Vegetation Index
def calculate_vi_model6(band_a, band_b, band_c, band_d): 
    vi_model6 = ((band_a - band_b) - 0.2 * (band_a - band_c)) * (band_c / band_b)
    return vi_model6
    # Modified Chlorophyll Absorption in Reflectance Index 1: MCARI1 = ((r700 - r670) - 0.2 * (r700 - r550)) * (r700 / r670)

# Model 7 in Vegetation Index
def calculate_vi_model7(band_a, band_b, band_c, band_d): 
    vi_model7 = 1.2 * (2.5 * (band_a - band_b) - 1.3 * (band_a - band_c))
    return vi_model7
    # Modified Chlorophyll Absorption in Reflectance Index 2: MCARI2 = 1.2 * (2.5 * (r800 - r670) - 1.3 * (r800 - r550))

# Model 8 in Vegetation Index
def calculate_vi_model8(band_a, band_b, band_c, band_d): 
    vi_model8 = 1.5 * (2.5 * (band_a - band_b) - 1.3 * (band_a - band_c)) / ((((2 * band_a + 1) ** 2) - (6 * band_a - 5 * (band_b ** 0.5)) - 0.5) ** 0.5)
    return vi_model8
    # Modified Chlorophyll Absorption in Reflectance Index 3: MCARI3 = 1.5 * (2.5 * (r800 - r670) - 1.3 * (r800 - r550)) / ((((2 * r800 + 1) ** 2) - (6 * r800 - 5 * (r670 ** 0.5)) - 0.5) ** 0.5)

# Model 9 in Vegetation Index
def calculate_vi_model9(band_a, band_b, band_c, band_d): 
    vi_model9 = 3 * ((band_a - band_b) - 0.2 * (band_a - band_c) * (band_a / band_b))
    return vi_model9
    # Transformed CARI: TCARI = 3 * ((r700 - r670) - 0.2 * (r700 - r550) * (r700 / r670))

# Model 10 in Vegetation Index
def calculate_vi_model10(band_a, band_b, band_c, band_d): 
    vi_model10 = 0.5 * (120 * (band_a - band_b) - 200 * (band_c - band_b))
    return vi_model10
    # Triangular Vegetation Index: TVI = 0.5 * (120 * (r750 - r550) - 200 * (r670 - r550))

# Model 11 in Vegetation Index
def calculate_vi_model11(band_a, band_b, band_c, band_d): 
    vi_model11 = 1.2 * (1.2 * (band_a - band_b) - 2.5 * (band_c - band_b))
    return vi_model11
    # Modified Triangular Vegetation Index 1: MTVI1 = 1.2 * (1.2 * (r800 - r550) - 2.5 * (r670 - r550))

# Model 12 in Vegetation Index
def calculate_vi_model12(band_a, band_b, band_c, band_d): 
    vi_model12 = (1.5 * (1.2 * (band_a - band_b) - 2.5 * (band_c - band_b))) / ((((2 * band_a + 1) ** 2 - (6 * band_a - (5 * (band_c ** 0.5)))) - 0.5) ** 0.5)
    return vi_model12
    # Modified Triangular Vegetation Index 2: MTVI2 = 1.5 * (1.2 * (r800 - r550) - 2.5 * (r670 - r550)) / ((((2 * r800 + 1) ** 2 - (6 * r800 - (5 * (r670 ** 0.5)))) - 0.5) ** 0.5)

# Model 13 in Vegetation Index
def calculate_vi_model13(band_a, band_b, band_c, band_d): 
    vi_model13 = 0.5 * (2 * band_a + 1 - (((2 * band_a + 1) ** 2) - 8 * (band_a - band_b)) ** 0.5)
    return vi_model13
    # Improved SAVI with self-adjustment factor L: MSAVI = 0.5 * (2 * r800 + 1 - (((2 * r800 + 1) ** 2) - 8 * (r800 - r670)) ** 0.5)

# Model 14 in Vegetation Index
def calculate_vi_model14(band_a, band_b, band_c, band_d): 
    vi_model14 = 2.5 * ((band_a - band_b) / (band_a + 2.4 * band_b + 1))
    return vi_model14
    # Enhanced Vegetation Index 2: EVI2 = 2.5 * ((r800 - r670) / (r800 + 2.4 * r670 + 1))

# Model 15 in Vegetation Index
def calculate_vi_model15(band_a, band_b, band_c, band_d): 
    vi_model15 = ((band_a - band_b)/(band_a + band_b)) - ((band_a - band_c)/(band_a + band_c))
    return vi_model15
    # Combine Mangrove Recognition Index (CMRI = NDVI - NDWI): CMRI = ((r865 - r660)/(r865 + r660)) - ((r857 - r1241)/(r857 + r1241))

# Model 16 in Vegetation Index
def calculate_vi_model16(band_a, band_b, band_c, band_d): 
    vi_model16 = ((band_a + 1.5 * band_b) - band_c) / (band_a - band_d)
    return vi_model16
    # Temperature Condition Index: TCI = ((r887 + 1.5 * r524) - r678) / (r887 - r701)

# Model 17 in Vegetation Index
def calculate_vi_model17(band_a, band_b, band_c, band_d): 
    vi_model17 = (band_a - band_b) / (band_a + band_b + 0.01)
    return vi_model17
    # Normalized Red Green Difference Vegetation Index: NDIG = (r650 - r560)/(r650 + r560 + 0.01)
    # Normalized Red Blue Difference Vegetation Index: NDIB = (r650 - r485)/(r650 + r485 + 0.01)

# Model 18 in Vegetation Index
def calculate_vi_model18(band_a, band_b, band_c, band_d): 
    vi_model18 = 1.5 * (band_a - band_b) / (band_a + band_b + 0.5)
    return vi_model18
    # Soil-adjusted Vegetation Index With Green: SAVIGRE = 1.5 * (r865 - r560) / (r865 + r560 + 0.5)

# Model 19 in Vegetation Index
def calculate_vi_model19(band_a, band_b, band_c, band_d): 
    vi_model19 = (band_a - band_b) / (band_a + band_b - band_c)
    return vi_model19
    # Visible Atmospherically Resistant Index: VARI2 = (r560 - r650) / (r560 + r650 - r485)
    
# Model 20 in Vegetation Index
def calculate_vi_model20(band_a, band_b, band_c, band_d): 
    vi_model20 = (band_a - band_b + 1.7 * (band_c - band_d)) /(band_a + band_b - 1.7 * (band_c - band_d))
    return vi_model20
    # Green Atmospherically Resistant Index: GARI = (r865 - r560 + 1.7 * (r485 - r650)) /(r865 + r560 - 1.7 * (r485 - r650))

# Model 21 in Vegetation Index
def calculate_vi_model21(band_a, band_b, band_c, band_d): 
    vi_model21 = (band_a) / (0.666 * band_b + 0.334 * band_c)
    return vi_model21
    # Carbon Dioxide Continuum Interpolated Band Ratio: CO2-CIBR = (r2010) / (0.666 * r1990 + 0.334 * r2040)

# Model 22 in Vegetation Index
def calculate_vi_model22(band_a, band_b, band_c, band_d): 
    vi_model22 = band_a / (band_b + band_c)
    return vi_model22
    # VI1 = r712 / (r998 + r693)

# Model 23 in Vegetation Index
def calculate_vi_model23(band_a, band_b, band_c, band_d): 
    vi_model23 = (band_a - band_b) / band_c
    return vi_model23
    # VI3 = (r708 - r693) / r758
    # Plant Senescence Reflectance Index: PSRI = (r660 - r554) / (r729)

# Model 24 in Vegetation Index
def calculate_vi_model24(band_a, band_b, band_c, band_d): 
    vi_model24 = (1 / band_a) - (1 / band_b)
    return vi_model24
    # VI6 = (1 / r1000) - (1 / r713)
    # Anthocyanin Reflectance Index 1: ARI1 =  (1 / r554) - (1 / r699)
    # Carotenoid Reflectance Index 1: CRI1 =  (1 / r507) - (1 / r546)
    # Carotenoid Reflectance Index 2: CRI2 =  (1 / r507) - (1 / r699)

# Model 25 in Vegetation Index
def calculate_vi_model25(band_a, band_b, band_c, band_d): 
    vi_model25 = (band_a / band_b) - 1
    return vi_model25
    # Chlorophyll Index red-edge: Clre = (r780 / r699) - 1

# Model 26 in Vegetation Index
def calculate_vi_model26(band_a, band_b, band_c, band_d): 
    vi_model26 = (band_a - band_b) / (band_a + band_b - 2 * band_c)
    return vi_model26
    # Normalized Difference red-edge 1 modified: NDre1m = (r729 - r699) / (r729 + r699 - 2 * r441)
    # Normalized Difference red-edge 2 modified: NDre2m = (r780 - r699) / (r780 + r699 - 2 * r441)

# Model 27 in Vegetation Index
def calculate_vi_model27(band_a, band_b, band_c, band_d): 
    vi_model27 = ((band_a / band_b) - 1 ) / ((band_a / band_b) + 1) ** 0.5
    return vi_model27
    # Modified Simple Ratio red-edge: MSRre = ((r833 / r699) - 1 ) / ((r833 / r699) + 1) ** 0.5
    # Modified Simple Ratio red-edge narrow: MSRren = ((r865 / r699) - 1 ) / ((r865 / r699) + 1) ** 0.5

# Model 28 in Vegetation Index
def calculate_vi_model28(band_a, band_b, band_c, band_d): 
    vi_model28 = (band_a - (band_b - (band_c - band_b))) / (band_a + (band_b - (band_c - band_b)))
    return vi_model28
    # Atmospherically Resistant Vegetation Index: ARVI = (r801 - (r679 - (r449 - r679))) / (r801 + (r679 - (r449 - r679)))

# Model 29 in Vegetation Index
def calculate_vi_model29(band_a, band_b, band_c, band_d): 
    vi_model29 = (band_a - band_b) / (band_a + band_b - 2 * band_c)
    return vi_model29
    # Modified Red Edge Normalized Difference Vegetation Index: MRENDVI = (r749 - r708) / (r749 + r708 - 2 * r441)

# Model 30 in Vegetation Index
def calculate_vi_model30(band_a, band_b, band_c, band_d): 
    vi_model30 = (np.log(1/band_a) - np.log(1/band_b)) / (np.log(1/band_a) + np.log(1/band_b))
    return vi_model30
    # Normalized Difference Nitrogen Index: NDNI =  (log(1/r1512) - log(1/r1677)) / (log(1/r1512) + log(1/r1677))
    # Normalized Difference Lignin Index: NDLI =  (log(1/r1755) - log(1/r1677)) / (log(1/r1755) + log(1/r1677))

# Model 31 in Vegetation Index
def calculate_vi_model31(band_a, band_b, band_c, band_d): 
    vi_model31 = 0.5 * (band_a + band_b) - band_c
    return vi_model31
    # Cellulose Absorption Index: CAI =  0.5 * (r1993 + r2206) - r2102

# Model 32 in Vegetation Index
def calculate_vi_model32(band_a, band_b, band_c, band_d): 
    vi_model32 = 100 * ((band_a - band_b) + (band_a - band_c))
    return vi_model32
    # Lignin Cellulose Absorption Index: LCAI =  100 * ((r2206 - r2159) + (r2206 - r2335))

# Model 33 in Vegetation Index
def calculate_vi_model33(band_a, band_b, band_c, band_d): 
    vi_model33 = band_a * ((1 / band_b) - (1 / band_c))
    return vi_model33
    # Anthocyanin Reflectance Index 2: ARI2 =  r801 * ((1 / r554) - (1 / r699))

# Model 34 in Vegetation Index
def calculate_vi_model34(band_a, band_b, band_c, band_d): 
    vi_model34 = (band_a - (band_b - band_c)) / (band_a + (band_b - band_c))
    return vi_model34
    # Normalized Multi-band Drought Index: NMDI = (r855 - (r1636 - r2127)) / (r855 + (r1636 - r2127))

In [ ]:
# # NEW VEGETATION HYPERSPECTRAL INDEX
# def calculate_new_vi_hyp_veg(band_a, band_b, band_c, band_d, band_e, band_f, band_g):
#     new_vi_hyp_veg = - 44*band_a + 23*band_b - 19*band_c + 52*band_d - 36*band_e
#     return new_vi_hyp_veg
#     # New Vegetation Hyperspectral Index: new_vi_hyp_veg = - 44*r929 + 23*r750 - 19*r1976 + 52*r2103 - 36*r2199

# def calculate_new_vi_hyp_soil(band_a, band_b, band_c, band_d, band_e, band_f, band_g):
#     new_vi_hyp_soil = 89*band_a - 94*band_b - 40*band_c + 47*band_d - 13*band_e + 44*band_f - 43*band_g
#     return new_vi_hyp_soil
#     # New Vegetation Hyperspectral Index: new_vi_hyp_soil = 89*r1099 - 94*r1078 - 40*r2238 + 47*r2261 - 13*r1976 + 44*r2103 - 43*r2199

# def calculate_new_vi_hyp_site(band_a, band_b, band_c, band_d, band_e, band_f, band_g):
#     new_vi_hyp_site = 108*band_a - 56*band_b - 39*band_c + 62*band_d - 83*band_e
#     return new_vi_hyp_site
#     # New Vegetation Hyperspectral Index: new_vi_hyp_site = 108*r781 - 56*r1078 - 39*r760 + 62*r1110 - 83*r877

In [ ]:
# def select_model_2(vi, band_a, band_b, band_c, band_d, band_e, band_f, band_g):
#     model_new_1 = ["NewHypVeg"]
#     model_new_2 = ["NewHypSoil"]
#     model_new_3 = ["NewHypSite"]

#     if vi in model_new_1:
#         veg_index_values = calculate_new_vi_hyp_veg(band_a, band_b, band_c, band_d, band_e, band_f, band_g)
#     elif vi in model_new_2:
#         veg_index_values = calculate_new_vi_hyp_soil(band_a, band_b, band_c, band_d, band_e, band_f, band_g)
#     elif vi in model_new_3:
#         veg_index_values = calculate_new_vi_hyp_site(band_a, band_b, band_c, band_d, band_e, band_f, band_g)
#     return veg_index_values

In [ ]:
def select_model(vi, band_a, band_b, band_c, band_d):
    model1 = ["G", "SRPI", "Lic2", "Ctr1", "Ctr2", "Vog1", "GM1", "GM2", "ZM", "FRI1", "FRI2", "SR", "WI1", "WI2", "MSI", "RVI", "GRVI", "GR", "GB", "P1", "P2", "VI8","WBI"]
    model2a = ["NPQI", "PRI1", "PRI2", "NPCI", "Lic1", "VARI", "NDWI", "NDWI2", "DWI", "HFDI", "VI7", "NDVIre1", "NDVIre1n", "NDVIre2", "NDVIre2n", "NDVIre3", "NDVIre3n", "NDre1", "NDre2","LWVI1","LWVI2","NDII"]
    model2b = ["SIPI", "MTCI", "LCI"]
    model2c = ["MVI"]
    model2d = ["Vog2", "Vog3","DWSI"]
    model2e = ["VI5"]
    model2f = ["SRre1","SRre2","SIPI1"]
    model3a = ["RDVI", "RDVI2", "RDVIREG"]
    model3b = ["MSR", "MSRREG"]
    model4 = ["MSR2"]
    model5a = ["OSAVI", "SAVI", "OSAVIREG"]
    model5b = ["GSAVI"]
    model6 = ["MCARI1"]
    model7 = ["MCARI2"]
    model8 = ["MCARI3"]
    model9 = ["TCARI"]
    model10 = ["TVI"]
    model11 = ["MTVI1"]
    model12 = ["MTVI2"]
    model13 = ["MSAVI"]
    model14 = ["EVI2"]
    model15 = ["CMRI"]
    model16 = ["TCI"]
    model17 = ["NDIG", "NDIB"]
    model18 = ["SAVIGRE"]
    model19 = ["VARI2"]
    model20 = ["GARI"]
    model21 = ["CO2-CIBR"]
    model22 = ["VI1"]
    model23 = ["VI3", "PSRI"]
    model24 = ["VI6","ARI1","CRI1","CRI2"]
    model25 = ["Clre"]
    model26 = ["NDre1m", "NDre2m"]
    model27 = ["MSRre", "MSRren"]
    model28 = ["ARVI"]
    model29 = ["MRENDVI"]
    model30 = ["NDNI", "NDLI"]
    model31 = ["CAI"]
    model32 = ["LCAI"]
    model33 = ["ARI2"]
    model34 = ["NMDI"]

    if vi in model1:
        veg_index_values = calculate_vi_model1(band_a, band_b, band_c, band_d)
    elif vi in model2a:
        veg_index_values = calculate_vi_model2a(band_a, band_b, band_c, band_d)
    elif vi in model2b:
        veg_index_values = calculate_vi_model2b(band_a, band_b, band_c, band_d)
    elif vi in model2c:
        veg_index_values = calculate_vi_model2c(band_a, band_b, band_c, band_d)
    elif vi in model2d:
        veg_index_values = calculate_vi_model2d(band_a, band_b, band_c, band_d)
    elif vi in model2e:
        veg_index_values = calculate_vi_model2e(band_a, band_b, band_c, band_d)
    elif vi in model2f:
        veg_index_values = calculate_vi_model2f(band_a, band_b, band_c, band_d)
    elif vi in model3a:
        veg_index_values = calculate_vi_model3a(band_a, band_b, band_c, band_d)
    elif vi in model3b:
        veg_index_values = calculate_vi_model3b(band_a, band_b, band_c, band_d)
    elif vi in model4:
        veg_index_values = calculate_vi_model4(band_a, band_b, band_c, band_d)
    elif vi in model5a:
        veg_index_values = calculate_vi_model5a(band_a, band_b, band_c, band_d)
    elif vi in model5b:
        veg_index_values = calculate_vi_model5b(band_a, band_b, band_c, band_d)
    elif vi in model6:
        veg_index_values = calculate_vi_model6(band_a, band_b, band_c, band_d)
    elif vi in model7:
        veg_index_values = calculate_vi_model7(band_a, band_b, band_c, band_d)
    elif vi in model8:
        veg_index_values = calculate_vi_model8(band_a, band_b, band_c, band_d)
    elif vi in model9:
        veg_index_values = calculate_vi_model9(band_a, band_b, band_c, band_d)
    elif vi in model10:
        veg_index_values = calculate_vi_model10(band_a, band_b, band_c, band_d)
    elif vi in model11:
        veg_index_values = calculate_vi_model11(band_a, band_b, band_c, band_d)
    elif vi in model12:
        veg_index_values = calculate_vi_model12(band_a, band_b, band_c, band_d)
    elif vi in model13:
        veg_index_values = calculate_vi_model13(band_a, band_b, band_c, band_d)
    elif vi in model14:
        veg_index_values = calculate_vi_model14(band_a, band_b, band_c, band_d)
    elif vi in model15:
        veg_index_values = calculate_vi_model15(band_a, band_b, band_c, band_d)
    elif vi in model16:
        veg_index_values = calculate_vi_model16(band_a, band_b, band_c, band_d)
    elif vi in model17:
        veg_index_values = calculate_vi_model17(band_a, band_b, band_c, band_d)
    elif vi in model18:
        veg_index_values = calculate_vi_model18(band_a, band_b, band_c, band_d)
    elif vi in model19:
        veg_index_values = calculate_vi_model19(band_a, band_b, band_c, band_d)
    elif vi in model20:
        veg_index_values = calculate_vi_model20(band_a, band_b, band_c, band_d)
    elif vi in model21:
        veg_index_values = calculate_vi_model21(band_a, band_b, band_c, band_d)
    elif vi in model22:
        veg_index_values = calculate_vi_model22(band_a, band_b, band_c, band_d)
    elif vi in model23:
        veg_index_values = calculate_vi_model23(band_a, band_b, band_c, band_d)
    elif vi in model24:
        veg_index_values = calculate_vi_model24(band_a, band_b, band_c, band_d)
    elif vi in model25:
        veg_index_values = calculate_vi_model25(band_a, band_b, band_c, band_d)
    elif vi in model26:
        veg_index_values = calculate_vi_model26(band_a, band_b, band_c, band_d)
    elif vi in model27:
        veg_index_values = calculate_vi_model27(band_a, band_b, band_c, band_d)
    elif vi in model28:
        veg_index_values = calculate_vi_model28(band_a, band_b, band_c, band_d)
    elif vi in model29:
        veg_index_values = calculate_vi_model29(band_a, band_b, band_c, band_d)
    elif vi in model30:
        veg_index_values = calculate_vi_model30(band_a, band_b, band_c, band_d)
    elif vi in model31:
        veg_index_values = calculate_vi_model31(band_a, band_b, band_c, band_d)
    elif vi in model32:
        veg_index_values = calculate_vi_model32(band_a, band_b, band_c, band_d)
    elif vi in model33:
        veg_index_values = calculate_vi_model33(band_a, band_b, band_c, band_d)
    elif vi in model34:
        veg_index_values = calculate_vi_model34(band_a, band_b, band_c, band_d)
    return veg_index_values

#### Dictionaries of vegetation indices bands

In [ ]:
# Classed bands to calculate vegetatio index
vi_classed_band_dict = {
    "NDVI": ("Red", "NIR"),
    "NBR": ("NIR", "SWIR 2"),
    "NDMI": ("NIR", "SWIR 1"),
    "DVI": ("Red", "NIR"),
    "DVIGRE": ("Green", "NIR"),
    "DVIRED": ("Red Edge", "NIR"),
    "CIREDGE": ("Red Edge", "NIR"),
    "CIGREEN": ("Green", "NIR"),
    "IPVI": ("Red", "NIR"),
    "NIRV": ("Red", "NIR"),
    "MNLI": ("NIR", "Red"),
    "NLI": ("NIR", "Red"),
    "WDRVI": ("NIR", "Red"),
    "NDRE": ("Red Edge", "NIR"),
    "BAI": ("Red Edge", "NIR"),
    "SVI": ("Red Edge", "Red"),
    "MRESR": ("Red Edge", "NIR"),
}

vi_classed_band_dict2 = {
    "TBDVI": ("Red", "NIR", "SWIR 1"),
    "EVI": ("Blue", "Red", "NIR"),
    "EXG2": ("Blue", "Green", "Red"),
    "NRI": ("Blue", "Green", "Red"),
    "NGI": ("Blue", "Green", "Red"),
    "GNDVI": ("Green", "Red Edge", "NIR"),
    "ENDVI": ("Red Edge", "Green", "Blue"),
    "MRGBVI": ("Red Edge", "Green", "Blue"),
    "NREI": ("Green", "Red Edge", "NIR"),
    "EXG": ("Red", "Green", "Blue"),
    "EXB": ("Red", "Green", "Blue"),
    "EXR": ("Red", "Green", "Blue"),
    "RGR": ("Red", "Green", "Blue"),
    "BGR": ("Red", "Green", "Blue"),
    "NGRDI": ("Red", "Green", "Blue"),
    "NGBDI": ("Red", "Green", "Blue"),
    "MGRVI": ("Red", "Green", "Blue"),
    "RGBVI": ("Red", "Green", "Blue"),
    "GLI": ("Red", "Green", "Blue"),
    "CIVE": ("Red", "Green", "Blue"),
    "EXGR": ("Red", "Green", "Blue"),
    "IRGBVI": ("Red", "Green", "Blue"),
    "REP": ("Red", "Red Edge", "NIR"),
}
# Fixed bands to calculate vegetation index
vi_fixed_band_dict = {
    "Vog1": (37, 35, 0, 0),
    "ZM": (38, 34, 0, 0),
    "Vog2": (37, 38, 35, 36),
    "Vog3": (37, 38, 35, 35),
    "MCARI1": (33, 30, 16, 0),
    "TCARI": (33, 30, 16, 0),
    "P1": (2, 189, 0, 0),
    "P2": (3, 190, 0, 0),
    "CO2-CIBR": (134, 133, 135, 0),
    "VI3": (34, 32, 39, 0),
    "VI3": (12, 10, 0, 0),

    # "NewHypVeg": (55, 38, 131, 142, 154, 0, 0),
    # "NewHypSoil": (75, 73, 159, 162, 131, 142, 154),
    # "NewHypSite": (41, 73, 39, 76, 50, 0, 0),
}
# Variable bands to calculate vegetation index
vi_variable_band_dict = {
    "G": (17, 31, 0, 0),
    "SRPI": (1, 31, 0, 0),
    "Lic2": (2, 32, 0, 0),
    "Ctr1": (33, 1, 0, 0),
    "Ctr2": (33, 39, 0, 0),
    "GM1": (38, 17, 0, 0),
    "GM2": (38, 33, 0, 0),
    "FRI1": (37, 43, 0, 0),
    "FRI2": (32, 22, 0, 0),
    "SR": (43, 30, 0, 0),
    "WI1": (52, 59, 0, 0),
    "WI2": (112, 45, 0, 0),
    "NPQI": (112, 45, 0, 0),
    "PRI1": (14, 19, 0, 0),
    "NPCI": (31, 1, 0, 0),
    "Lic1": (43, 31, 0, 0),
    "VARI": (16, 30, 0, 0),
    "NDWI": (43, 112, 0, 0),
    "NDWI2": (48, 88, 0, 0),
    "DWI": (44, 156, 0, 0),
    "SIPI": (43, 3, 28, 0),
    "RDVI": (43, 30, 0, 0),
    "MSR2": (43, 30, 0, 0),
    "OSAVI": (43, 30, 0, 0),
    "MCARI2": (43, 30, 16, 0),
    "MCARI3": (43, 30, 16, 0),

    "TVI": (38, 16, 30, 0),
    "MTVI1": (43, 16, 30, 0),
    "MTVI2": (43, 16, 30, 0),
    "MSAVI": (43, 30, 0, 0),
    "EVI2": (43, 30, 0, 0),
    "MSI": (112, 45, 0, 0),
    "MVI": (47, 18, 113, 0),
    "CMRI": (49, 29, 88, 0),
    "SAVI": (51, 31, 0, 0),
    "RVI": (51, 31, 0, 0),
    "TCI": (51, 13, 31, 33),
    "GRVI": (49, 16, 0, 0),
    "GR": (16, 28, 0, 0),
    "GB": (16, 8, 0, 0),
    "MTCI": (49, 35, 28, 0),
    "MVI": (47, 18, 113, 0),
    "RDVI2": (49, 30, 0, 0),
    "RDVIREG": (49, 35, 0, 0),
    "MSR": (49, 29, 0, 0),
    "MSRREG": (49, 35, 0, 0),
    "MSR2": (43, 30, 0, 0),
    "OSAVIREG": (49, 35, 0, 0),
    "NDIG": (28, 18, 0, 0),
    "NDIB": (28, 8, 0, 0),
    "SAVIGRE": (49, 18, 0, 0),
    "VARI2": (18, 28, 0, 0),
    "GARI": (49, 18, 8, 28),
    "GSAVI": (43, 18, 0, 0),
    "HFDI": (185, 137, 0, 0),
    "VI7": (65, 34, 0, 0),
    "VI1": (34, 65, 32, 0),
    "VI6": (65, 34, 0, 0),
    "VI8": (65, 34, 0, 0),

    "NDVIre1": (46, 33, 0, 0),
    "NDVIre1n": (49, 33, 0, 0),
    "NDVIre2": (46, 36, 0, 0),
    "NDVIre2n": (49, 36, 0, 0),
    "NDVIre3": (46, 41, 0, 0),
    "NDVIre3n": (49, 41, 0, 0),
    "NDre1": (36, 33, 0, 0),
    "NDre2": (41, 33, 0, 0),
    "LCI": (48, 34, 31, 0),
    "PSRI": (29, 17, 36, 0),
    "Clre": (41, 33, 0, 0),
    "NDre1m": (36, 33, 2, 0),
    "NDre2m": (41, 33, 2, 0),
    "SRre1": (36, 2, 33, 0),
    "SRre2": (36, 2, 33, 0),
    "MSRre": (46, 33, 0, 0),
    "MSRren": (49, 33, 0, 0),
    "SIPI1": (2, 43, 30, 0),
    "WBI": (59, 52, 0, 0),
    "LWVI1": (75, 64, 0, 0),
    "LWVI2": (75, 85, 0, 0),
    "NDII": (45, 117, 0, 0),
    "DWSI": (43, 16, 118, 50),
    "ARI1": (17, 33, 0, 0),
    "CRI1": (11, 16, 0, 0),
    "CRI2": (11, 33, 0, 0),
    "ARVI": (43, 31, 3, 0),
    "MRENDVI": (38, 34, 2, 0),
    "NDNI": (104, 120, 0, 0),
    "NDLI": (128, 120, 0, 0),
    "CAI": (133, 155, 142, 0),
    "LCAI": (155, 149, 172, 0),
    "ARI2": (43, 17, 33, 0),
    "NMDI": (48, 116, 145, 0),
}

#### Read the excel file

In [ ]:
# Read the Excel file
excel_file = working_dir + "CulebraPointsCBI.xlsx"
points = pd.read_excel(excel_file)

In [ ]:
# Initialize lists to store results
cb_results_lst = 0
cb2_results_lst = 0
fb_results_lst = 0
vb_results_lst = 0
rb_results_lst = 0

### FOR CLASSED BANDS

#### Combination of bands of CLASSED BANDS

In [ ]:
def bands_combinations(tiff_file, band_lst, vi_classed_band_dict):
    for i in vi_classed_band_dict:
        print(i)
    vi_name = input("Insert the index name: ")
    combination_lst = []
    band_names = vi_classed_band_dict[vi_name]
    with rasterio.open(tiff_file) as src:
        for band_a in band_lst:
            if band_a[1] == band_names[0]:
                band_ax = band_a[0]
                for band_b in band_lst:
                    if band_b[1] == band_names[1]:
                        band_bx = band_b[0]
                        combination_lst.append((band_ax, band_bx))
    for i in combination_lst:
        print(i)
    return combination_lst, vi_name

def bands_combinations2(tiff_file, band_lst, vi_classed_band_dict2):
    for i in vi_classed_band_dict2:
        print(i)
    vi_name = input("Insert the index name: ")
    combination_lst2 = []
    band_names = vi_classed_band_dict2[vi_name]
    with rasterio.open(tiff_file) as src:
        for band_a in band_lst:
            if band_a[1] == band_names[0]:
                band_ax = band_a[0]
                for band_b in band_lst:
                    if band_b[1] == band_names[1]:
                        band_bx = band_b[0]
                        for band_c in band_lst:
                            if band_c[1] == band_names[2]:
                                band_cx = band_c[0]
                                combination_lst2.append((band_ax, band_bx, band_cx))
    for i in combination_lst2:
        print(i)
    return combination_lst2, vi_name

#### Calculate vegetation index values for CLASSED BANDS

In [ ]:
def get_vi_cb_image(tiff_file, vi_name, band_a, band_b):
    with rasterio.open(tiff_file) as dataset:
        band_1 = dataset.read(band_a)
        band_2 = dataset.read(band_b)
        if vi_name == "NDVI":
            vi_values = calculate_ndvi(band_1, band_2)
        elif vi_name == "NBR":
            vi_values = calculate_nbr(band_1, band_2)
        elif vi_name == "NDMI":
            vi_values = calculate_ndmi(band_1, band_2)
        elif vi_name == "DVI":
            vi_values = calculate_dvi(band_1, band_2)
        elif vi_name == "DVIGRE":
            vi_values = calculate_dvigre(band_1, band_2)
        elif vi_name == "DVIRED":
            vi_values = calculate_dvired(band_1, band_2)
        elif vi_name == "CIREDGE":
            vi_values = calculate_ciredge(band_1, band_2)
        elif vi_name == "CIGREEN":
            vi_values = calculate_cigreen(band_1, band_2)
        elif vi_name == "IPVI":
            vi_values = calculate_ipvi(band_1, band_2)
        elif vi_name == "NIRV":
            vi_values = calculate_nirv(band_1, band_2)
        elif vi_name == "MNLI":
            vi_values = calculate_mnli(band_1, band_2)
        elif vi_name == "NLI":
            vi_values = calculate_nli(band_1, band_2)
        elif vi_name == "WDRVI":
            vi_values = calculate_wdrvi(band_1, band_2)
        elif vi_name == "NDRE":
            vi_values = calculate_ndre(band_1, band_2)
        elif vi_name == "BAI":
            vi_values = calculate_bai(band_1, band_2)
        elif vi_name == "SVI":
            vi_values = calculate_svi(band_1, band_2)
        elif vi_name == "MRESR":
            vi_values = calculate_mresr(band_1, band_2)
        else:
            print("Insert other vegetation index name")
    return vi_values, dataset

def get_vi_cb_at_points(vi_values, dataset, points):
    veg_index_values = []
    for _, row in points.iterrows():
        name, x, y = row['Parcela'], row['Coord_X'], row['Coord_Y']
        row_idx, col_idx = rasterio.transform.rowcol(dataset.transform, x, y)
        veg_index_value = vi_values[row_idx, col_idx]
        veg_index_values.append((name, veg_index_value))
    return veg_index_values

def get_cb_comparative_points(tiff_file, band_lst, vi_classed_band_dict, points):
    cb_results_lst = []
    combination_lst, vi_name = bands_combinations(tiff_file, band_lst, vi_classed_band_dict)
    for paired_bands in combination_lst:
        band_a = paired_bands[0]
        band_b = paired_bands[1]
        vi_values, dataset = get_vi_cb_image(tiff_file, vi_name, band_a, band_b)
        veg_index_values = get_vi_cb_at_points(vi_values, dataset, points)
        cb_results_lst.append((vi_name, band_a, band_b, veg_index_values))
    return cb_results_lst

In [ ]:
def get_vi_cb2_image(tiff_file, vi_name, band_a, band_b, band_c):
    with rasterio.open(tiff_file) as dataset:
        band_1 = dataset.read(band_a)
        band_2 = dataset.read(band_b)
        band_3 = dataset.read(band_c)
        if vi_name == "TBDVI":
            vi_values = calculate_tbdvi(band_1, band_2, band_3)
        elif vi_name == "EVI":
            vi_values = calculate_evi(band_1, band_2, band_3)
        elif vi_name == "EXG2":
            vi_values = calculate_exg2(band_1, band_2, band_3)
        elif vi_name == "NRI":
            vi_values = calculate_nri(band_1, band_2, band_3)
        elif vi_name == "NGI":
            vi_values = calculate_ngi(band_1, band_2, band_3)
        elif vi_name == "GNDVI":
            vi_values = calculate_gndvi(band_1, band_2, band_3)
        elif vi_name == "ENDVI":
            vi_values = calculate_endvi(band_1, band_2, band_3)
        elif vi_name == "MRGBVI":
            vi_values = calculate_mrgbvi(band_1, band_2, band_3)
        elif vi_name == "NREI":
            vi_values = calculate_nrei(band_1, band_2, band_3)
        elif vi_name == "EXG":
            vi_values = calculate_exg(band_1, band_2, band_3)
        elif vi_name == "EXB":
            vi_values = calculate_exb(band_1, band_2, band_3)
        elif vi_name == "EXR":
            vi_values = calculate_exr(band_1, band_2, band_3)
        elif vi_name == "RGR":
            vi_values = calculate_rgr(band_1, band_2, band_3)
        elif vi_name == "BGR":
            vi_values = calculate_bgr(band_1, band_2, band_3)
        elif vi_name == "NGRDI":
            vi_values = calculate_ngrdi(band_1, band_2, band_3)
        elif vi_name == "NGBDI":
            vi_values = calculate_ngbdi(band_1, band_2, band_3)
        elif vi_name == "MGRVI":
            vi_values = calculate_mgrvi(band_1, band_2, band_3)
        elif vi_name == "RGBVI":
            vi_values = calculate_rgbvi(band_1, band_2, band_3)
        elif vi_name == "GLI":
            vi_values = calculate_gli(band_1, band_2, band_3)
        elif vi_name == "CIVE":
            vi_values = calculate_cive(band_1, band_2, band_3)
        elif vi_name == "EXGR":
            vi_values = calculate_exgr(band_1, band_2, band_3)
        elif vi_name == "IRGBVI":
            vi_values = calculate_irgbvi(band_1, band_2, band_3)
        elif vi_name == "REP":
            vi_values = calculate_rep(band_1, band_2, band_3)
        else:
            print("Insert other vegetation index name")
    return vi_values, dataset

def get_cb2_comparative_points(tiff_file, band_lst, vi_classed_band_dict2, points):
    cb2_results_lst = []
    combination_lst2, vi_name = bands_combinations2(tiff_file, band_lst, vi_classed_band_dict2)
    for paired_bands in combination_lst2:
        band_a = paired_bands[0]
        band_b = paired_bands[1]
        band_c = paired_bands[2]
        vi_values, dataset = get_vi_cb2_image(tiff_file, vi_name, band_a, band_b, band_c)
        veg_index_values = get_vi_cb_at_points(vi_values, dataset, points)
        cb2_results_lst.append((vi_name, band_a, band_b, band_c, veg_index_values))
    return cb2_results_lst

#### Extract results of wavelength and vegetation index of CLASSED BANDS

In [ ]:
cb_results_lst = get_cb_comparative_points(tiff_file, band_lst, vi_classed_band_dict, points)
for i in cb_results_lst:
    print(i)

In [ ]:
cb2_results_lst = get_cb2_comparative_points(tiff_file, band_lst, vi_classed_band_dict2, points)
for i in cb2_results_lst:
    print(i)

### FOR FIXED BANDS

#### Calculate vegetation index values for FIXED BANDS

In [ ]:
def get_vi_fb_image(tiff_file, vi_fixed_band_dict):
    vi_list = []
    for vi in vi_fixed_band_dict:
        band = vi_fixed_band_dict[vi]
        band = tuple(1 if b == 0 else b for b in band)
        with rasterio.open(tiff_file) as dataset:
            band_a = dataset.read(band[0])
            band_b = dataset.read(band[1])
            band_c = dataset.read(band[2])
            band_d = dataset.read(band[3])

        veg_index_values = select_model(vi, band_a, band_b, band_c, band_d)
        vi_list.append((veg_index_values, dataset))
    return vi_list

def get_vi_fb_at_points(veg_index, dataset, points):
    veg_index_values = []
    for _, row in points.iterrows():
        name, x, y = row['Parcela'], row['Coord_X'], row['Coord_Y']
        row_idx, col_idx = rasterio.transform.rowcol(dataset.transform, x, y)
        veg_index_value = veg_index[row_idx, col_idx]
        veg_index_values.append((name, veg_index_value))
    return veg_index_values

def get_fb_comparative_points(tiff_file, vi_fixed_band_dict, points):
    fb_results_lst = []
    
    vi_list = get_vi_fb_image(tiff_file, vi_fixed_band_dict)

    iv_keys = []
    for key in vi_fixed_band_dict:
        iv_keys.append(key)
    cnt = 0

    for tup in vi_list:
        veg_index = tup[0]
        dataset = tup[1]
        veg_index_values = get_vi_fb_at_points(veg_index, dataset, points)

        band_tup = vi_fixed_band_dict[iv_keys[cnt]]
        fb_results_lst.append((iv_keys[cnt], band_tup[0], band_tup[1], band_tup[2], band_tup[3], veg_index_values))
        cnt += 1
    return fb_results_lst

#### Extract results of wavelength and vegetations index of FIXED BANDS

In [ ]:
fb_results_lst = get_fb_comparative_points(tiff_file, vi_fixed_band_dict, points)
for i in fb_results_lst:
    print(i)

### FOR VARIABLE BANDS

#### Combination of bands of VARIABLE BANDS

In [ ]:
import itertools

def get_vb_combination_bands(vi_variable_band_dict):
    vi_combination_lst = []
    
    for key, val_tup in vi_variable_band_dict.items():
        total_neighborhood = [[] for _ in range(len(val_tup))]  # Create a list of empty lists for each value in the tuple
        for idx, val in enumerate(val_tup):
            if val != 0:
                band_neighborhood = [val + i for i in range(-2, 3)]  # Generate values from val-2 to val+2
            else:
                band_neighborhood = [val]  # If the value is zero, keep it as zero in all combinations
            total_neighborhood[idx] = band_neighborhood
        
        # Generate all possible combinations from the neighborhoods
        combinations = list(itertools.product(*total_neighborhood))
        
        # Add the key to each combination and filter out combinations with negative values
        for combination in combinations:
            if all(x >= 0 for x in combination):
                vi_combination_lst.append((key, *combination))
    
    return vi_combination_lst

#### Calculate vegetation index values for VARIABLE BANDS

In [ ]:
def get_vi_vb_image(tiff_file, vi_variable_band_dict):
    vi_list = []
    
    vb_combination_band = get_vb_combination_bands(vi_variable_band_dict)
    for val_tup in vb_combination_band:
        val_tup = tuple(1 if b == 0 else b for b in val_tup)
        with rasterio.open(tiff_file) as dataset:
            band_a = dataset.read(val_tup[1])
            band_b = dataset.read(val_tup[2])
            band_c = dataset.read(val_tup[3])
            band_d = dataset.read(val_tup[4])

        veg_index_values = select_model(val_tup[0], band_a, band_b, band_c, band_d)
        vi_list.append((veg_index_values, dataset))
    return vi_list, vb_combination_band

def get_vi_vb_at_points(veg_index, dataset, points):
    veg_index_values = []
    for _, row in points.iterrows():
        name, x, y = row['Parcela'], row['Coord_X'], row['Coord_Y']
        row_idx, col_idx = rasterio.transform.rowcol(dataset.transform, x, y)
        veg_index_value = veg_index[row_idx, col_idx]
        veg_index_values.append((name, veg_index_value))
    return veg_index_values

def get_vb_comparative_points(tiff_file, vi_variable_band_dict, points):
    vb_results_lst = []
    
    vi_list, vb_combination_band = get_vi_vb_image(tiff_file, vi_variable_band_dict)

    count = 0

    for tup in vi_list:
        veg_index = tup[0]
        dataset = tup[1]
        veg_index_values = get_vi_vb_at_points(veg_index, dataset, points)

        band_tup = vb_combination_band[count]
        vb_results_lst.append((band_tup[0], band_tup[1], band_tup[2], band_tup[3], band_tup[4],veg_index_values))
        count += 1
    return vb_results_lst

####  Extract results of wavelength and vegetation index of VARIABLE BANDS

In [ ]:
vb_results_lst = get_vb_comparative_points(tiff_file, vi_variable_band_dict, points)
# for i in vb_results_lst:
#     print(i)

### FOR RAW BANDS

#### Calculate vegetation index values for RAW BANDS

In [ ]:
def get_rb_image(tiff_file, band_number):
    with rasterio.open(tiff_file) as dataset:
        band_values = dataset.read(band_number)
    return band_values, dataset

def get_rb_at_points(band_values, dataset, points):
    band_values_lst = []
    for _, row in points.iterrows():
        name, x, y = row['Parcela'], row['Coord_X'], row['Coord_Y']
        row_idx, col_idx = rasterio.transform.rowcol(dataset.transform, x, y)
        band_value = band_values[row_idx, col_idx]
        band_values_lst.append((name, band_value))
    return band_values_lst

def get_rb_comparative_points(tiff_file, band_lst, points):
    rb_results_lst = []

    for band_number in range(1,192):
        band_values, dataset = get_rb_image(tiff_file, band_number)
        band_values_lst = get_rb_at_points(band_values, dataset, points)
        rb_results_lst.append((band_lst[band_number-1][1], band_number, band_lst[band_number-1][2], band_values_lst))

    return rb_results_lst

#### Extract results of wavelength and vegetation index of RAW BANDS

In [ ]:
rb_results_lst = get_rb_comparative_points(tiff_file, band_lst, points)
# for i in rb_results_lst:
#     print(i)

### SAVE INDICES VALUES

#### Save vegetation indices values in an excel file (.xlsx)

In [ ]:
def save_xlsx(input_file, results_lst):
    # Creamos un diccionario para almacenar los datos
    data_dict = {}

    for elm in results_lst:
        if results_lst == cb_results_lst:
            col_name = f"{elm[0]}_{elm[1]}_{elm[2]}"
            type_name = "classed_bands"
        elif results_lst == cb2_results_lst:
            col_name = f"{elm[0]}_{elm[1]}_{elm[2]}_{elm[3]}"
            type_name = "classed_bands"
        elif results_lst == fb_results_lst:
            col_name = f"{elm[0]}_{elm[1]}_{elm[2]}_{elm[3]}_{elm[4]}"
            type_name = "fixed_bands"
        elif results_lst == vb_results_lst:
            col_name = f"{elm[0]}_{elm[1]}_{elm[2]}_{elm[3]}_{elm[4]}"
            type_name = "variable_bands"
        elif results_lst == rb_results_lst:
            col_name = f"{elm[0]}_{elm[1]}_{elm[2]:.0f}nm"
            type_name = "raw_bands"
        data_dict[col_name] = [value for point, value in elm[len(elm)-1] ]

    # Convertimos el diccionario en un DataFrame
    df = pd.DataFrame(data_dict)

    ### PARA DATAFRAMES CON MÁS DE 16000 COLUMNAS ###
    # Si el DataFrame tiene más de 16000 columnas, dividirlo en partes
    max_columns = 16000
    num_parts = (df.shape[1] // max_columns) + 1

    for part in range(num_parts):
        start_col = part * max_columns
        end_col = start_col + max_columns
        df_part = df.iloc[:, start_col:end_col]

        # Define the output file name for each part
        if type_name == "classed_bands":
            output_file = input_file[:-5] + f"_hyp_{type_name}_{results_lst[0][0]}_p{part+1}.xlsx"
        else:
            output_file = input_file[:-5] + f"_hyp_{type_name}_p{part+1}.xlsx"

        # Copia el archivo original al nuevo archivo
        shutil.copyfile(input_file, output_file)

        # Abre el nuevo archivo
        wb = openpyxl.load_workbook(output_file)
        sheet = wb.active

        # Encuentra la última columna con datos en la hoja existente
        last_col = sheet.max_column

        # Escribe los datos del DataFrame en las columnas nuevas a partir de la última columna existente
        for col_idx, col_name in enumerate(df_part.columns, start=last_col + 1):
            sheet.cell(row=1, column=col_idx, value=col_name)
            for row_idx, value in enumerate(df_part[col_name], start=2):
                sheet.cell(row=row_idx, column=col_idx, value=value)

        # Guarda el archivo con las nuevas columnas
        wb.save(output_file)

    # # Copia el archivo original al nuevo archivo
    # if type_name == "classed_bands":
    #     output_file = input_file[:-5] + "_hyp_" + type_name + "_" + results_lst[0][0] + ".xlsx"
    # else:
    #     output_file = input_file[:-5] + "_hyp_" + type_name + ".xlsx"
    # shutil.copyfile(input_file, output_file)

    # # Abre el nuevo archivo
    # wb = openpyxl.load_workbook(output_file)
    # sheet = wb.active

    # # Encuentra la última columna con datos en la hoja existente
    # last_col = sheet.max_column

    # # Escribe los datos del DataFrame en las columnas nuevas a partir de la última columna existente
    # for col_idx, col_name in enumerate(df.columns, start=last_col + 1):
    #     sheet.cell(row=1, column=col_idx, value=col_name)
    #     for row_idx, value in enumerate(df[col_name], start=2):
    #         sheet.cell(row=row_idx, column=col_idx, value=value)

    # # Guarda el archivo con las nuevas columnas
    # wb.save(output_file)
    return output_file

In [ ]:
def save_csv(input_file, results_lst):
    # Creamos un diccionario para almacenar los datos
    data_dict = {}

    for elm in results_lst:
        if results_lst == cb_results_lst:
            col_name = f"{elm[0]}_{elm[1]}_{elm[2]}"
            type_name = "classed_bands"
        elif results_lst == cb2_results_lst:
            col_name = f"{elm[0]}_{elm[1]}_{elm[2]}_{elm[3]}"
            type_name = "classed_bands"
        elif results_lst == fb_results_lst:
            col_name = f"{elm[0]}_{elm[1]}_{elm[2]}_{elm[3]}_{elm[4]}"
            type_name = "fixed_bands"
        elif results_lst == vb_results_lst:
            col_name = f"{elm[0]}_{elm[1]}_{elm[2]}_{elm[3]}_{elm[4]}"
            type_name = "variable_bands"
        elif results_lst == rb_results_lst:
            col_name = f"{elm[0]}_{elm[1]}_{elm[2]:.0f}nm"
            type_name = "raw_bands"
        data_dict[col_name] = [value for point, value in elm[len(elm)-1]]

    # Convertimos el diccionario en un DataFrame
    df = pd.DataFrame(data_dict)

    # Define the output file name
    if type_name == "classed_bands":
        output_file = input_file[:-5] + "_hyp_" + type_name + "_" + results_lst[0][0] + ".csv"
    else:
        output_file = input_file[:-5] + "_hyp_" + type_name + ".csv"

    # Save the DataFrame to a CSV file
    df.to_csv(output_file, index=False)
    return output_file


In [ ]:
## Options to save file
# save_xlsx(excel_file, cb_results_lst)
save_xlsx(excel_file, cb2_results_lst)
# save_xlsx(excel_file, fb_results_lst)
# save_xlsx(excel_file, vb_results_lst)
# save_xlsx(excel_file, rb_results_lst)

In [ ]:
save_csv(excel_file, cb2_results_lst)